# Figure 5: S and Cl behaviour during degassing

In [1]:
# import python packages
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import math

## Functions to calculate behaviour of Cl during degassing

Uses partion coefficient from Tattitch et al. (2021)

Tattitch, B., Chelle-Michou, C., Blundy, J. et al. Chemical feedbacks during magma degassing control chlorine partitioning and metal extraction in volcanic arcs. Nat Commun 12, 1774 (2021). https://doi.org/10.1038/s41467-021-21887-w

In [2]:
# Coefficients for eq. (1) from Tattitch et al. (2021)
def Cl_Kd_Tattich21_constants():
    D0 = 0.005654
    a = 0.05749
    b = 0.1099
    P0 = 172.8
    c = 40.42
    return D0,a,b,c,P0

# Cl partition coefficient from eq. (1) in Tattitch et al. (2021)
def Cl_Kd_Tattitch21(NaClf,SiO2m,P):
    # NaClf = wt%, SiO2m = wt%, P = MPa
    D0,a,b,c,P0 = Cl_Kd_Tattich21_constants()
    Kd = D0*((math.exp(a*NaClf+b*SiO2m))/(1. + math.exp((P0-P)/c)))
    return Kd

# Melt Cl concentration in ppmw for a given fluid salinity
def Cl_conc(NaClf,SiO2m,P):
    # NaClf = wt%, SiO2m = wt%, P = MPa
    Kd = Cl_Kd_Tattitch21(NaClf,SiO2m,P)
    Cl_ppm_m = ((35.453*NaClf)/(58.44)/Kd)*10000.
    return Cl_ppm_m

# Newton Raphson solver
def newton_raphson(x0,constants,e1,step,eqs,deriv,maxiter=100):
    
    def dx(x,eqs):
        f_ = eqs(x,constants)
        result =(abs(0-f_))
        return result
    
    def nr(x0,eqs,deriv):
        f_ = eqs(x0,constants)
        df_ = deriv
        x0 = x0 - step*(f_/df_)
        return x0
    
    i = 0.
    for iter in range(maxiter):
        i = i+1
        f_ = eqs(x0,constants)
        df_ = deriv(x0,constants)
        x0 = x0 - step*(f_/df_)
        delta1 = dx(x0,eqs)   
        if abs(delta1) < e1:
            return x0

# Calculate melt and vapor Cl concentration given fractions of total melt and vapor and total Cl content of the system
def Cl_distribution(inputs,guessx,nr_step=1,nr_tol=1e-6):
    
    D0,a,b,c,P0 = Cl_Kd_Tattich21_constants()

    constants = inputs['wClT'],inputs['wf'],inputs['P'],inputs['SiO2m'],D0,a,b,c,P0
   
    def distribution(wClf, constants):
        wClT,wf,P,SiO2m,D0,a,b,c,P0 = constants
        wClm = (wClT-(wf*wClf))/(1.-wf)
        return wClf,wClm

    def f(wClf, constants):
        wClT,wf,P,SiO2m,D0,a,b,c,P0 = constants
        NaClf = ((wClf/10000.)/35.453)*58.44
        Kd = Cl_Kd_Tattitch21(NaClf,SiO2m,P)
        wClm = wClf/Kd
        diff = wClT - (wf*wClf) - ((1.-wf)*wClm)
        return diff
    
    def df(wClf, constants):
        wClT,wf,P,SiO2m,D0,a,b,c,P0 = constants
        result = -wf + 0.000164837954474939*a*wClf*(1.0 - wf)*(math.exp((-P + P0)/c) + 1.0)*math.exp(-SiO2m*b - 0.000164837954474939*a*wClf)/D0 - (1.0 - wf)*(math.exp((-P + P0)/c) + 1.0)*math.exp(-SiO2m*b - 0.000164837954474939*a*wClf)/D0
        return result
    
    wClf = newton_raphson(guessx,constants,nr_tol,nr_step,f,df)
    result1 = distribution(wClf, constants)
    return result1

# Calculate Cl concentration of the melt and vapor for a given degassing path (i.e., fractions of melt and vapor) and initial Cl content of the system
def Cl_quasi_degas(wClT_ppm,SiO2m,input):
    P_start = input.loc[0,'pressure']
    results_headers = pd.DataFrame([["P","SiO2_melt","NaCl_fluid_wtpc","Cl_ppm_f",'w_fluid','w_melt',"Kd_Cl","Cl_ppm_m"]])
    results1 = pd.DataFrame([[P_start,SiO2m,0.,0.,0.,1.,"",wClT_ppm,]])
    results = pd.concat([results_headers, results1])
    guessx = 1.
    for n in range(1,len(input),1):
        P = input.loc[n,'pressure']
        wf = input.loc[n,'vapor_fraction']
        inputs = {'wClT':wClT_ppm,'wf':wf,'P':P,'SiO2m':SiO2m}
        wClf_ppm, wClm_ppm = Cl_distribution(inputs,guessx)
        guessx = wClf_ppm
        NaClf = ((wClf_ppm/10000.)/35.453)*58.44
        Kd = Cl_Kd_Tattitch21(NaClf,SiO2m,P)
        results1 = pd.DataFrame([[P,SiO2m,NaClf,wClf_ppm,wf,1.-wf,Kd,wClm_ppm]])
        results = pd.concat([results, results1])
    results.columns = results.iloc[0]
    results = results[1:]
    results.reset_index(inplace=True)  
    return results

## Degassing paths for H2O, CO2, and S using Sulfur_X

Ding, S., Plank, T., Wallace, P.J. and Rasmussen, D.J., 2023. Sulfur_X: A model of sulfur degassing during magma ascent. Geochemistry, Geophysics, Geosystems, 24(4), p.e2022GC010552. https://doi.org/10.1029/2022GC010552

In [3]:
# All assume 1000 ppm CO2, 1000 ppm S, and initial DFMQ of 0.

# Uses dry basalt composition from EoV_2-2_Fig4_sulfur
dry_basalt_degas = pd.read_csv('files/sulfur_X_dry_basalt.csv')
# Uses dry basalt composition from EoV_2-2_Fig4_sulfur but with 5 wt% H2O
wet_basalt_degas = pd.read_csv('files/sulfur_X_wet_basalt.csv')
# Uses wet andesite composition from EoV_2-2_Fig4_sulfur
wet_andesite_degas = pd.read_csv('files/sulfur_X_wet_andesite.csv')

## Degassing Cl using Sulfur_X degassing paths and Tattitch et al. (2021) Cl Kd

In [4]:
# All have 1000 ppm Cl initially
Cl_dry_basalt = Cl_quasi_degas(1000.,50.14,dry_basalt_degas)
Cl_wet_basalt = Cl_quasi_degas(1000.,50.14,wet_basalt_degas)
Cl_wet_andesite = Cl_quasi_degas(1000.,60.32,wet_andesite_degas)

## Plotting

In [5]:
fig = make_subplots(rows=1, cols=2, shared_yaxes = True, shared_xaxes = False,vertical_spacing=0.08, horizontal_spacing=0.05)

# define curve and symbol attributes
lw = 2
syms = 7
lc1 = '#800080' # purple, dry basalt
lc2 = '#FF8C03'  # orange, wet andesite
lc3 = 'black'
lc4 = 'lightgrey'
lc5 = 'darkgrey'
lc6 = '#0160C6' # blue wet basalt

r = 1
# P vs. conc.
c = 1
# S
# dry basalt
fig.add_trace(go.Scatter(mode = "lines", x=dry_basalt_degas['wS_melt'], y=dry_basalt_degas['pressure'], line_color = lc1, line_width = lw,line_dash = "solid"), row = r, col = c)
# wet basalt
fig.add_trace(go.Scatter(mode = "lines", x=wet_basalt_degas['wS_melt'], y=wet_basalt_degas['pressure'], line_color = lc6, line_width = lw,line_dash = "solid"), row = r, col = c)
# wet andesite
fig.add_trace(go.Scatter(mode = "lines", x=wet_andesite_degas['wS_melt'], y=wet_andesite_degas['pressure'], line_color = lc2, line_width = lw,line_dash = "solid"), row = r, col = c)
# Cl
# dry basalt
fig.add_trace(go.Scatter(mode = "lines", x=Cl_dry_basalt['Cl_ppm_m'], y=Cl_dry_basalt['P'], line_color = lc1, line_width = lw,line_dash = "dash"), row = r, col = c)
# wet basalt
fig.add_trace(go.Scatter(mode = "lines", x=Cl_wet_basalt['Cl_ppm_m'], y=Cl_wet_basalt['P'], line_color = lc6, line_width = lw,line_dash = "dash"), row = r, col = c)
# wet andesite
fig.add_trace(go.Scatter(mode = "lines", x=Cl_wet_andesite['Cl_ppm_m'], y=Cl_wet_andesite['P'], line_color = lc2, line_width = lw,line_dash = "dash"), row = r, col = c)
# axes
fig.update_yaxes(title='Pressure (MPa)',range=[450,0],dtick=100, row = r, col = c)
fig.update_xaxes(title = "S or Cl (ppm)",  range=[0,1100],dtick=200, row = r, col = c)

# P vs. Kd
c = 2
fig.add_trace(go.Scatter(mode = "lines", x=[1,1], y=[0,450], line_color = lc5, line_width = lw*0.6,line_dash = "dot"), row = r, col = c)
# S
# dry basalt
fig.add_trace(go.Scatter(mode = "lines", x=dry_basalt_degas['kd_combined_wt'], y=dry_basalt_degas['pressure'], line_color = lc1, line_width = lw,line_dash = "solid"), row = r, col = c)
# wet basalt
fig.add_trace(go.Scatter(mode = "lines", x=wet_basalt_degas['kd_combined_wt'], y=wet_basalt_degas['pressure'], line_color = lc6, line_width = lw,line_dash = "solid"), row = r, col = c)
# wet andesite
fig.add_trace(go.Scatter(mode = "lines", x=wet_andesite_degas['kd_combined_wt'], y=wet_andesite_degas['pressure'], line_color = lc2, line_width = lw,line_dash = "solid"), row = r, col = c)
# Cl
# dry basalt
fig.add_trace(go.Scatter(mode = "lines", x=Cl_dry_basalt['Kd_Cl'], y=Cl_dry_basalt['P'], line_color = lc1, line_width = lw*1.6,line_dash = "dash"), row = r, col = c)
# wet basalt
fig.add_trace(go.Scatter(mode = "lines", x=Cl_wet_basalt['Kd_Cl'], y=Cl_wet_basalt['P'], line_color = lc6, line_width = lw,line_dash = "dash"), row = r, col = c)
# wet andesite
fig.add_trace(go.Scatter(mode = "lines", x=Cl_wet_andesite['Kd_Cl'], y=Cl_wet_andesite['P'], line_color = lc2, line_width = lw,line_dash = "dash"), row = r, col = c)
# axes
fig.update_xaxes(title = "<i>K</i><sub>d</sub><sup>vapor-melt</sup>",  side='right', type='log', exponentformat='power', range=[-2,4], row = r, col = c)

# annotations
r=1
c=1
fig.add_annotation(x=50, y=420,text="(a)",showarrow=False, font=dict(size=15),textangle=0,  row = r, col = c)
fig.add_annotation(x=150, y=110,text="\u2014 dry basalt",showarrow=False,
                   ax = 0, ay = 0, font=dict(color=lc1), textangle=0, row = r, col = c)
fig.add_annotation(x=155, y=140,text="\u2014 wet basalt",showarrow=False,
                   ax = 0, ay = 0, font=dict(color=lc6), textangle=0, row = r, col = c)
fig.add_annotation(x=180, y=170,text="\u2014 wet andesite",showarrow=False,
                   ax = 0, ay = 0, font=dict(color=lc2), textangle=0, row = r, col = c)
fig.add_annotation(x=70, y=230,text="\u2014 S",showarrow=False,
                   ax = 0, ay = 0, font=dict(color='black'), textangle=0, row = r, col = c)
fig.add_annotation(x=70, y=260,text="\u2013 \u2013 Cl",showarrow=False,
                   ax = 0, ay = 0, font=dict(color='black'), textangle=0, row = r, col = c)
c=2
fig.add_annotation(x=math.log10(0.02), y=420,text="(b)",showarrow=False, font=dict(size=15),textangle=0,  row = r, col = c)
fig.add_annotation(x=math.log10(700), y=110,text="\u2014 dry basalt",showarrow=False,
                   ax = 0, ay = 0, font=dict(color=lc1), textangle=0, row = r, col = c)
fig.add_annotation(x=math.log10(740), y=140,text="\u2014 wet basalt",showarrow=False,
                   ax = 0, ay = 0, font=dict(color=lc6), textangle=0, row = r, col = c)
fig.add_annotation(x=math.log10(1000), y=170,text="\u2014 wet andesite",showarrow=False,
                   ax = 0, ay = 0, font=dict(color=lc2), textangle=0, row = r, col = c)
fig.add_annotation(x=math.log10(260), y=230,text="\u2014 S",showarrow=False,
                   ax = 0, ay = 0, font=dict(color='black'), textangle=0, row = r, col = c)
fig.add_annotation(x=math.log10(250), y=260,text="\u2013 \u2013 Cl",showarrow=False,
                   ax = 0, ay = 0, font=dict(color='black'), textangle=0, row = r, col = c)


fig.update_layout(height=400, width=800, plot_bgcolor='rgb(255,255,255)' , showlegend = False)

fig.update_xaxes(showline=True, linewidth=1, linecolor='black', mirror=True, ticks="inside", ticklen=5, title_standoff = 0, tickcolor="black", tickfont=dict(size=12))
fig.update_yaxes(showline=True, linewidth=1, linecolor='black', mirror=True, ticks="inside", ticklen=5, title_standoff = 0, tickcolor="black", tickfont=dict(size=12))

fig.update_layout(font_family="Arial",font_color="black")

fig.show()